In [1]:
from bn256 import *
import random
from copy import copy
from functools import reduce
import time

In [2]:
def timer(func):
    def _func(*args, **kwargs):
        t = time.clock()
        res = func(*args, **kwargs)
        t = time.clock() - t
        print('{0}: time elapsed {1:.8f} s.' .format(func.__name__, t))
        return res

    return _func

In [3]:
def hamminge_distance(s1, s2):
    l = len(s1)
    assert l == len(s2), "Undefined for sequences of unequal length."
    return sum(el1 != el2 for el1, el2 in zip(s1, s2)) / l

In [4]:
a = (-1, 1)
def gen_vector(size):
    return tuple(random.choice(a) for i in range(size))

In [5]:
v1, v2 = gen_vector(16), gen_vector(16)

In [6]:
print(v1, v2, sep="\n")

(1, 1, 1, -1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1)
(-1, 1, 1, -1, -1, -1, 1, -1, 1, 1, -1, 1, 1, -1, -1, -1)


In [7]:
def inverse_G1(curve):
    inv = copy(curve)
    inv.y = inv.y.additive_inverse()
    return inv

In [8]:
def inverse_G2(twist):
    inv = copy(twist)
    inv.negate()
    return inv

In [9]:
def transfer_vec_to_G1(vec):
    return tuple(inverse_G1(curve_G) if p == -1 else curve_G for p in vec)

In [10]:
def transfer_vec_to_G2(vec):
    return tuple(inverse_G2(twist_G) if p == -1 else twist_G for p in vec)

In [11]:
def gen_keys(size):
    return tuple(random.randrange(2, order) for i in range(size))

In [12]:
v1points = transfer_vec_to_G2(v1)
v2points = transfer_vec_to_G1(v2)

In [13]:
v1points

(((21167961636542580255011770066570541300993051739349375019639421053990175267184,64746500191241794695844075326670126197795977525365406531717464316923369116492), (20666913350058776956210519119118544732556678129809273996262322366050359951122,17778617556404439934652658462602675281523610326338642107814333856843981424549)),
 ((21167961636542580255011770066570541300993051739349375019639421053990175267184,64746500191241794695844075326670126197795977525365406531717464316923369116492), (20666913350058776956210519119118544732556678129809273996262322366050359951122,17778617556404439934652658462602675281523610326338642107814333856843981424549)),
 ((21167961636542580255011770066570541300993051739349375019639421053990175267184,64746500191241794695844075326670126197795977525365406531717464316923369116492), (20666913350058776956210519119118544732556678129809273996262322366050359951122,17778617556404439934652658462602675281523610326338642107814333856843981424549)),
 ((2116796163654258025501177006657054

In [14]:
v2points

((1, 2),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 2),
 (1, 2),
 (1, 2),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 2),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 2),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 2),
 (1, 2),
 (1, 2))

In [15]:
h1, h2 = random.randrange(2, order), random.randrange(2, order)
H1, H2 = curve_G.scalar_mul(h1), twist_G.scalar_mul(h2)

In [16]:
h1, h2

(22942342483393590154307636543123965345250070757065701566321760158081320119772,
 4369951427774527586922509095769360426127473581784410182547555374132153134022)

In [17]:
@timer
def master_key_generation():
    s, t = gen_keys(16), gen_keys(16)
    u, v = gen_keys(18), gen_keys(18)
    gen1_h = tuple(point_add(twist_G.scalar_mul(si), H2.scalar_mul(ti)) for si, ti in zip(s, t))
    gen2_h = tuple(point_add(curve_G.scalar_mul(ui), H1.scalar_mul(vi)) for ui, vi in zip(u, v))
    return gen1_h, s, t, gen2_h, u, v

In [18]:
gen1_h, s, t, gen2_h, u, v = master_key_generation()

master_key_generation: time elapsed 13.28954380 s.


In [19]:
gen1_h, s, t, gen2_h, u, v

((((36757837448597238728942167219838024990220063423513314558724913904193197658317,47812470917670948932502899591813501441544700253926011076049979666189996502429), (22952210370606111619911332483825431540134973013350134661584896475637415639900,18921031371722653061212647312571891204839722447188466454391646280040882529487)),
  ((35099111259376958531070856202011405167407773227124900661641434045813229182744,13749906784866460466363790843618164125115456082361911337021920709504678783981), (27624192741986256273261616319160159284829767635283376350255392158641464504779,20006333330674035675134734893122205843619070774130102630570031662338071470479)),
  ((17312983784128911915317756412756877398128456447827897861244570492816965874158,32257911813684461876517515320727913922785743619844915512829773623436471671667), (58223718856569034302055098398652797810610012688316666958953851152598537521719,43382639577477698977239139668019790581826631791285146974079547443558650923707)),
  ((306340234301956319889673860235

In [20]:
r0, r1 = random.randrange(2, order), random.randrange(2, order)

In [21]:
@timer
def decryption_key_generation(gen1_h, s, t, c):
    n = len(c)
    
    d01 = twist_G.scalar_mul(r0)
    d02 = H2.scalar_mul(r0)
    
    d03 = c[0].scalar_mul(s[0])
    d04 = c[0].scalar_mul(t[0])
    for i in range(1, n):
        d03 = point_add(d03, c[i].scalar_mul(s[i]))
        d04 = point_add(d04, c[i].scalar_mul(t[i]))
    d03 = point_add(inverse_G2(d03), gen1_h[0].scalar_mul(r0))
    d04 = point_add(inverse_G2(d04), gen1_h[1].scalar_mul(r0))
    
    DK = [d01, d02, d03, d04]
    for i in range(n):
        DK.append(point_add(c[i], gen1_h[i].scalar_mul(r0)))
    
    return tuple(DK)

In [22]:
DK = decryption_key_generation(gen1_h, s, t, v1points)

decryption_key_generation: time elapsed 13.51353994 s.


In [23]:
len(DK)

20

In [24]:
DK

(((31196603631840949985019373959467965302248726432674596514435116567773862177981,5755704248988315748910070533279138329025238107105512479428097044601897801871), (20312382275887047984319760551010187197375046933213646526116748139391062068533,31068295207432898178268823564194325909440231553757973893473477918073995888955)),
 ((7212646509760266738725787205891384092021507963910048585382961561553864117852,17684859305606430902153040634522486091704502241464070433503757534583887043648), (61668157268941961825964299831746367672549662318506239760905187638850174296443,40755831816684031367599883615284324184938028919951195623569941341758860101902)),
 ((51932560023258992204602892040077875111874561038160923962752053774368805539085,62964557961712274519760260846845254834092214782490251721015833161713126689562), (58125275558486277401332281435298806472183326983448956267316692300851594209951,63130596496940746221521437950979362780557971497148482259259105603740374487190)),
 ((441953485589989058602843071249315364

In [25]:
@timer
def encryption_authentication(gen2_h, u, v, c):
    n = len(c)
    
    c03 = curve_G.scalar_mul(r0)
    c04 = H1.scalar_mul(r0)
    
    CT0 = [c03, c04]
    for i in range(n):
        CT0.append(point_add(c[i], gen2_h[i].scalar_mul(r0)))
        
    c01 = c03.scalar_mul(u[0])
    c02 = c04.scalar_mul(v[0])
    for i in range(1, n + 2):
        c01 = point_add(c01, CT0[i].scalar_mul(u[i]))
        c02 = point_add(c02, CT0[i].scalar_mul(v[i]))
    
    c01 = inverse_G1(c01)
    c02 = inverse_G1(c02)
    
    return (c01, c02) + tuple(CT0)

In [26]:
CT = encryption_authentication(gen2_h, u, v, v2points)

encryption_authentication: time elapsed 3.28255286 s.


In [27]:
len(CT)

20

In [28]:
CT

((56461984004506773216689516270519140185112354685347561047159204960492690653742, 2478727406998786870726387739573954719681733193956607720491105099911747162123),
 (14630707499703302156494870663091028026940683818250939728216203338629579888527, 25337199954643902887399565710726941730475171199306206829037436436519268084708),
 (60467375863984284550499714511479722612774558931824753878501542956127337864179, 15834357423957270665058884366743903578437636375988966096802797460646640334158),
 (24593223306766891100671571299590489432451027545575812559941750051865299418954, 828764282627968314611200834568088247637160424206637898240799947124033335233),
 (56583592597294236830762397294776817528347234571901969537896883783891364441749, 55449333557190928275867874330803350581000304556974515467966161402329444631867),
 (24009302948635231793938856111592560344319820749476887190588699890500466874307, 26085006396777087173624979546381746954008240402792256364817973054777456734332),
 (11380024416636693319974282273521480

In [29]:
CT_v1 = encryption_authentication(gen2_h, u, v, transfer_vec_to_G1(v1))
logarithmic_table = [optimal_ate(dki, cki) for dki, cki in zip(DK, CT_v1)]

encryption_authentication: time elapsed 4.44119016 s.


In [30]:
logarithmic_table

[(((28380611984409812118010667536730463170333480233054722863417543027604119988736,3724574662029321284796646807074895158380710867040341548859203508758507731373),(48426820643701893592977584756070396133543985301205328845052950233680184152427,45853404831721465675467283737670981024876608177215505525672797352844537751819),(50355391743216735922763204318529109331044762222186189859055948050998493353138,44872606032480262076290402956305443986605270666493807049899279460229255461419)),((2170248604123878323645725868210396359611539470660549863994212004831624858079,14815960795229310957618696329241542475895895551549765444704051624761872899834),(383213109207297487085504929094092839280024437556580563713904875890291286181,18540907042035547753232787750402511588042290438517030188988798512006668261441),(55343292676139396914960545839166939635339484760779836822603887054344238524701,63831573342545902027409343643511101163749326607023167175239012274031575621082))),
 (((43511083826398104379668220775436236748609593

In [31]:
@timer
def decryption_authentication(DK, CT):
    return [optimal_ate(dki, cti) for dki, cti in zip(DK, CT)]

In [32]:
DA = decryption_authentication(DK, CT)

decryption_authentication: time elapsed 15.87139821 s.


In [33]:
DA

[(((50864595948795583100440691533551546037868422644783109681881590923448669284887,23474918770076635712644232341761014305598338373045176541413453162737753363008),(41844988071288237403917117740081536632002783857208960726195079238417095361079,15607074444156363696330994931093354060487573093227028168199002543408544166399),(36628717600857751014018979885747686506598741824010855163275644027152248322483,33189532246227435168073486427851215226447331121685281788907150924455944668801)),((16498992173451173971172707925128305153603501170092818161594125740137446859681,11579841035023595523774102570894527933752702090916829721948700782309972370130),(15444485066134462645230069232755222833850510847774289478018595715777630303840,18618250980874839016875046814872317572809116615662806260294128220157776618770),(57994552407400554793056820020746850434372859539357761098635701037965479588683,196296157970821369677971491428474741234504140113819330113691970909771910052))),
 (((458650986020671648859889564100944766626779

In [34]:
len(logarithmic_table), len(DA)

(20, 20)

In [35]:
hamminge_distance(logarithmic_table[4:], DA[4:])

0.4375

In [36]:
hamminge_distance(v1, v2)

0.4375